# Induced gap and magnetic field

# TO-DO:
* Add tunnel barrier, could use something like:

```python
def hoppingkind_at_interface(hop, shape1, shape2, syst):
    """Returns an HoppingKind iterator for hoppings at an interface between
       shape1 and shape2."""
    def at_interface(site1, site2, shape1, shape2):
        return ((shape1[0](site1.pos) and shape2[0](site2.pos)) or
                (shape2[0](site1.pos) and shape1[0](site2.pos)))
    hoppingkind = kwant.HoppingKind(hop.delta, hop.family_a)(syst)
    return ((i, j) for (i, j) in hoppingkind
            if at_interface(i, j, shape1, shape2))
```

In [ ]:
# 1. Standard library imports
from functools import lru_cache, partial
from types import SimpleNamespace

# 2. External package imports
import holoviews as hv
import kwant
from kwant.continuum.discretizer import discretize
from kwant.digest import uniform
import matplotlib.pyplot as plt
import numpy as np
from scipy.constants import hbar, m_e, eV, physical_constants, e
import sympy
from sympy.physics.quantum import TensorProduct as kr

# 3. Internal imports
# nothing here yet

hv.notebook_extension()

sx, sy, sz = [sympy.physics.matrices.msigma(i) for i in range(1, 4)]
s0 = sympy.eye(2)

# Parameters taken from arXiv:1204.2792
# All constant parameters, mostly fundamental constants, in a SimpleNamespace.
constants = SimpleNamespace(
    m_eff=0.015 * m_e,  # effective mass in kg
    hbar=hbar,
    m_e=m_e,
    eV=eV,
    e=e,
    meV=eV * 1e-3,
    c=1e18 / (eV * 1e-3))  # to get to meV * nm^2

constants.t = (hbar ** 2 / (2 * constants.m_eff)) * constants.c
constants.mu_B = physical_constants['Bohr magneton'][0] / constants.meV

# Hamiltonian and system definition

In [ ]:
def discretized_hamiltonian(a):
    k_x, k_y, k_z = kwant.continuum.momentum_operators
    x, y, z = kwant.continuum.position_operators
    B_x, B_y, B_z, Delta, mu, alpha, g, mu_B, hbar = sympy.symbols(
        'B_x B_y B_z Delta mu alpha g mu_B hbar', real=True)
    m_eff, mu_sc, mu_sm = sympy.symbols(
        'm_eff, mu_sc, mu_sm', commutative=False)
    c = sympy.symbols('c')  # should be (1e18 / constants.meV) if in nm and meV
    kin = (1 / 2) * hbar**2 * (k_x**2 + k_y**2 + k_z**2) / m_eff * c

    ham = ((kin - mu) * kr(s0, sz) +
           alpha * (k_y * kr(sx, sz) - k_x * kr(sy, sz)) +
           0.5 * g * mu_B * (B_x * kr(sx, s0) + B_y * kr(sy, s0) + B_z * kr(sz, s0)) +
           Delta * kr(s0, sx))

    args = dict(lattice_constant=a)
    subs_sm = [(Delta, 0), (mu, mu_sm)]
    subs_sc = [(g, 0), (alpha, 0), (mu, mu_sc)]

    templ_sm = discretize(ham.subs(subs_sm), **args)
    templ_sc = discretize(ham.subs(subs_sc), **args)
    
    return templ_sm, templ_sc


def add_disorder_to_template(template):
    def onsite_dis(site, p):
        identity = np.eye(4)
        return p.disorder * (uniform(repr(site), repr(p.salt)) - .5) * identity

    if onsite_disorder:
        first_site = next(iter(template.sites()))
        onsite = template[first_site]
        template[first_site] = lambda s, p: onsite(s, p) + onsite_dis(s, p)

    return template


def peierls(func, a, direction):
    def phase(s1, s2, p):
        x, y, z = s1.tag
        A = [p.B_y * z - p.B_z * y, 0, p.B_x * y][direction]
        A *= a**2 * 1e-18 * constants.e / constants.hbar
        return np.exp(-1j * A)

    def with_phase(s1, s2, p):
        hop = func(s1, s2, p)
        phi = phase(s1, s2, p)
        if p.orbital:
            if hop.shape[0] == 2:
                hop *= phi
            elif hop.shape[0] == 4:
                hop *= np.array([phi, phi.conj(), phi,
                                 phi.conj()], dtype='complex128')
        return hop
    return with_phase


def apply_peierls_to_template(template):
    """Adds p.orbital argument to the hopping functions."""
    for (site1, site2), hop in template.hopping_value_pairs():
        direction = np.argmax(site2.tag - site1.tag)
        a = max(max(site1.pos, site2.pos))
        template[site1, site2] = peierls(hop, a, direction)
    return template

# Shape functions

In [ ]:
def square_sector(r_out, r_in=0, L=1, L0=0, phi=360, angle=0, a=10):
    """Returns the shape function and start coords of a wire
    with a square cross section, for -r_out <= x, y < r_out.

    Parameters
    ----------
    r_out : int
        Outer radius in nm.
    r_in : int
        Inner radius in nm.
    L : int
        Length of wire from L0 in nm, -1 if infinite in x-direction.
    L0 : int
        Start position in x.
    phi : ignored
        Ignored variable, to have same arguments as cylinder_sector.
    angle : ignored
        Ignored variable, to have same arguments as cylinder_sector.
    a : int
        Discretization constant in nm.

    Returns
    -------
    (shape_func, *(start_coords))
    """
    r_in /= 2
    r_out /= 2 
    if r_in > 0:
        def shape(site):
            x, y, z = site.pos
            shape_yz = -r_in <= y < r_in and r_in <= z < r_out
            return (shape_yz and L0 <= x < L) if L > 0 else shape_yz
        return shape, lat.closest((L - a, 0, r_in + a))
    else:
        def shape(site):
            x, y, z = site.pos
            shape_yz = -r_out <= y < r_out and -r_out <= z < r_out
            return (shape_yz and L0 <= x < L) if L > 0 else shape_yz
        return shape, (L - a, 0, 0)


def cylinder_sector(r_out, r_in=0, L=1, L0=0, phi=360, angle=0, a=10):
    """Returns the shape function and start coords for a wire with
    as cylindrical cross section.

    Parameters
    ----------
    r_out : int
        Outer radius in nm.
    r_in : int, optional
        Inner radius in nm.
    L : int, optional
        Length of wire from L0 in nm, -1 if infinite in x-direction.
    L0 : int, optional
        Start position in x.
    phi : int, optional
        Coverage angle in degrees.
    angle : int, optional
        Angle of tilting from top in degrees.
    a : int, optional
        Discretization constant in nm.

    Returns
    -------
    (shape_func, *(start_coords))
    """
    phi *= np.pi / 360
    angle *= np.pi / 180
    r_out_sq, r_in_sq = r_out**2, r_in**2

    def shape(site):
        x, y, z = site.pos
        n = (y + 1j * z) * np.exp(1j * angle)
        y, z = n.real, n.imag
        rsq = y**2 + z**2
        shape_yz = r_in_sq <= rsq < r_out_sq and z >= np.cos(phi) * np.sqrt(rsq)
        return (shape_yz and L0 <= x < L) if L > 0 else shape_yz
    
    r_mid = (r_out + r_in) / 2
    start_coords = np.array([L - a, 
                             r_mid * np.sin(angle), 
                             r_mid * np.cos(angle)])

    return shape, np.int64(np.round(start_coords / a))

# System construction

In [ ]:
@lru_cache()
def make_3d_wire(a, L, r1, r2, phi, angle, onsite_disorder,
                 with_leads, with_shell, shape):
    """Create a cylindrical 3D wire covered with a
    superconducting (SC) shell, but without superconductor in 
    leads.

    Parameters
    ----------
    a : int
        Discretization constant in nm.
    L : int
        Length of wire (the scattering part with SC shell.)
    r1 : int
        Radius of normal part of wire in nm.
    r2 : int
        Radius of superconductor in nm.
    phi : int
        Coverage angle of superconductor in degrees.
    angle : int
        Angle of tilting of superconductor from top in degrees.
    onsite_disorder : bool
        When True, disorder in SM and requires `disorder` and `salt` aguments.
    with_leads : bool
        If True it adds infinite semiconducting leads.
    with_shell : bool
        Adds shell to the scattering area. If False no SC shell is added and
        only a cylindrical or square wire will be created.
    shape : str
        Either `circle` or `square` shaped cross section.

    Returns
    -------
    syst : kwant.builder.FiniteSystem
        The finilized kwant system.

    Examples
    --------
    This doesn't use default parameters because the variables need to be saved,
    to a file. So I create a dictionary that is passed to the function.

    >>> syst_params = dict(a=10, angle=0, site_disorder=False,
    ...                    L=30, phi=185, r1=50, r2=70, shape='square',
    ...                    with_leads=True, with_shell=True)
    >>> syst, hopping = make_3d_wire(**syst_params)

    """
    templ_sm, templ_sc = map(apply_peierls_to_template, discretized_hamiltonian(a))
    symmetry = kwant.TranslationalSymmetry((a, 0, 0))
    syst = kwant.Builder()
    lat = next(iter(templ_sc.sites()))[0]

    if shape == 'square':
        shape_function = square_sector
    elif shape == 'circle':
        shape_function = cylinder_sector
    else:
        raise(NotImplementedError('Only square or circle wire cross section allowed'))

    shape_normal = shape_function(r_out=r1, angle=angle, L=L, a=a)
    shape_normal_lead = shape_function(r_out=r1, angle=angle, L=-1, a=a)
    shape_sc = shape_function(r_out=r2, r_in=r1, phi=phi, angle=angle, L0=0, L=L, a=a)

    if onsite_disorder:
        templ_sm = add_disorder_to_template(templ_sm)

    syst.fill(templ_sm, *shape_normal)

    if with_shell:
        syst.fill(templ_sc, *shape_sc)

    if with_leads:
        cons_law = np.kron(np.eye(2), -np.array(sz).astype(np.float))
        lead = kwant.Builder(symmetry, conservation_law=cons_law)
        lead.fill(templ_sm, *shape_normal_lead)
        syst.attach_lead(lead)
        syst.attach_lead(lead.reversed())
    return syst.finalized()


def make_lead(a, r1, r2, phi, angle, with_shell, shape):
    """Create an infinite cylindrical 3D wire partially covered with a
    superconducting (SC) shell.

    Parameters
    ----------
    a : int
        Discretization constant in nm.
    r1 : int
        Radius of normal part of wire in nm.
    r2 : int
        Radius of superconductor in nm.
    phi : int
        Coverage angle of superconductor in degrees.
    angle : int
        Angle of tilting of superconductor from top in degrees.
    with_shell : bool
        Adds shell to the scattering area. If False no SC shell is added and
        only a cylindrical or square wire will be created.
    shape : str
        Either `circle` or `square` shaped cross section.

    Returns
    -------
    syst : kwant.builder.InfiniteSystem
        The finilized kwant system.

    Examples
    --------
    This doesn't use default parameters because the variables need to be saved,
    to a file. So I create a dictionary that is passed to the function.

    >>> syst_params = dict(a=10, angle=0, phi=185, r1=50,
    ...                    r2=70, shape='square', with_shell=True)
    >>> syst, hopping = make_lead(**syst_params)

    """
    templ_sm, templ_sc = map(apply_peierls_to_template, discretized_hamiltonian(a))

    lat = next(iter(templ_sc.sites()))[0]
    symmetry = kwant.TranslationalSymmetry((a, 0, 0))

    if shape == 'square':
        shape_function = square_sector
    elif shape == 'circle':
        shape_function = cylinder_sector
    else:
        raise(NotImplementedError('Only square or circle wire cross section allowed'))

    shape_normal_lead = shape_function(r_out=r1, angle=angle, L=-1, a=a)
    shape_sc_lead = shape_function(r_out=r2, r_in=r1, phi=phi, angle=angle, L=-1, a=a)

    lead = kwant.Builder(symmetry)
    lead.fill(templ_sm, *shape_normal_lead)
    if with_shell:
        lead.fill(templ_sc, *shape_sc_lead)
    return lead.finalized()

# Physics functions

In [ ]:
from adaptive_sampling import sample_function
def conductance(syst, p, E=100e-3, verbose=False):
    """Conductance is N - R_ee + R_he"""
    smatrix = kwant.smatrix(syst, energy=E, args=[p, 1e-6])
    r_ee = smatrix.submatrix((0, 0), (0, 0))
    r_eh = smatrix.submatrix((0, 0), (0, 1))
    r_hh = smatrix.submatrix((0, 1), (0, 1))
    r_he = smatrix.submatrix((0, 1), (0, 0))
    c = lambda r: np.trace(r @ r.T.conj()).real

    if verbose:
        r_ = {'r_ee': c(r_ee), 'r_eh': c(r_eh), 'r_he': c(r_he), 'r_hh': c(r_hh)}
        for key, val in r_.items():
            print('{val}: {key}'.format(val=val, key=key))

    N_e = r_ee.shape[0]

    return N_e - c(r_ee) + c(r_he)


def bands(lead, p, ks=None):
    bands = kwant.physics.Bands(lead, args=[p])
    
    if ks is None:
        bands = sample_function(np.vectorize(bands), [-3, 3])
        
    return np.array([bands(k) for k in ks])

# Usage

In [ ]:
ham_pars = dict(alpha=20, B_x=0, B_y=0, B_z=0, Delta=0, g=50, mu_B=constants.mu_B,
                orbital=True, c=constants.c, hbar=constants.hbar,
                m_eff=constants.m_eff, mu_sc=0, mu_sm=0)

p = SimpleNamespace(**ham_pars)

syst_params = dict(a=10, angle=45, onsite_disorder=False,
                   L=200, phi=135, r1=50, r2=80, shape='circle',
                   with_leads=True, with_shell=True)
syst = make_3d_wire(**syst_params)
kwant.plot(syst)

In [ ]:
%%opts Path [aspect='square']
lead = make_lead(10, 50, 70, 135, 0, True, 'circle')
p.B_x = 1
p.B_y = 1
p.orbital = True
ks = np.linspace(-3, 3)
Es = bands(lead, p, ks)
p1 = hv.Path((ks, Es))[:, -100:100]

p.orbital = False
Es1 = bands(lead, p, ks)
p2 = hv.Path((ks, Es1))[:, -100:100]

p1 * p2

In [ ]:
ham_pars = dict(alpha=20, B_x=0, B_y=0, B_z=0, Delta=0., g=50, mu_B=constants.mu_B,
                orbital=True, c=constants.c, hbar=constants.hbar,
                m_eff=constants.m_e, mu_sc=0, mu_sm=0)

p = SimpleNamespace(**ham_pars)
S = kwant.smatrix(syst, args=[p])